# Gráficos para la Web con Bokeh

La biblioteca Bokeh (http://bokeh.pydata.org/) permite crear visualizaciones en Python, pero orientadas a su utilización en navegadores Web. El objetivo es conseguir:
* visualizaciones interactivas al estilo de D3.js pero permitiendo su programación en Python
* y dando soporte a streams de datos.

Dado que un Notebook de Jupyter utiliza HTML para visualizar los resultados, Bokeh se integra con los Notebooks, como puede verse en los [ejemplos de bokeh](http://nbviewer.ipython.org/github/bokeh/bokeh-notebooks/blob/master/index.ipynb).

Bokeh se inspira en la notación [Grammar of Graphics de Wilkinson](https://www.cs.uic.edu/~wilkinson/TheGrammarOfGraphics/GOG.html), al igual que el paquete [ggplot2](http://ggplot2.org/) de R, pero no son compatibles.

Para instalarlo, se puede utilizar pip o conda. Si tenemos Anaconda lo mejor es:
```
conda install bokeh
```

Es fundamental entender que Bokeh es una interfaz que utiliza JavaScript en el navegador para la visualización, por lo que la programación se puede hacer en otros lenguajes que no son Python, por ejemplo en [R](https://hafen.github.io/rbokeh/#background). 
![Bokeh](http://i1.wp.com/www.analyticsvidhya.com/wp-content/uploads/2015/08/Bokeh_Intro.png?zoom=2&resize=398%2C339)

## Los tres niveles en Bokeh

Bokeh se puede utilizar en tres niveles:
* low-level *bokeh.models* interface, máxima flexibilidad, pero hay que hacerlo todo parte a parte.
* intermediate-level *bokeh.plotting* interface, centrado en la composición de glifos visuales (glyphs).
* high-level *bokeh.charts* interface que simplifican la tarea de hacer visualizaciones o gráficos complejos, tomándolos de un catálogo pre-construido.


## Bokeh como paleta de componentes

Bokeh puede utilizad como salidas diferentes formatos, entre ellos el del propio Notebook.

In [1]:
from bokeh.io import output_notebook, show
output_notebook() # Configura el Notebook como salida por defecto.

Loading BokehJS ...

Si utilizamos la interfaz **charts** solo tenemos que importar el tipo de gráfico que queramos. 

Por ejemplo, vamos a cargar el dataset de películas de IMDB que está en el paquete ggplot2 de R (pueden consultarse [aquí](http://vincentarelbundock.github.io/Rdatasets/)).

In [3]:
#import statsmodels.api as sm
#imdb = sm.datasets.get_rdataset("movies", "ggplot2")
#print imdb.__doc__
import pandas as pd

data = pd.read_csv("movie_metadata.csv")
data.head()


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [7]:
from bokeh.charts import Scatter
samplesize = 200 # With the full dataset it becomes slow.
tools = "pan, box_zoom, reset, resize" # omit the annoying "wheel_zoom"

p = Scatter(data[:samplesize], x="duration", y="imdb_score", 
            color="color", tools=tools, legend="top_right")
show(p)

Podemos ahora observar las valoraciones de acuerdo a la clasificación [MPAA](https://en.wikipedia.org/wiki/Motion_Picture_Association_of_America_film_rating_system). Pero podemos observar que hay muchos valores nulos, que tenemos que excluir del análisis.

In [10]:
import numpy as np
print np.unique(data["content_rating"])
print data.groupby("content_rating")["movie_title"].count()

[nan 'Approved' 'G' 'GP' 'M' 'NC-17' 'Not Rated' 'PG' 'PG-13' 'Passed' 'R'
 'TV-14' 'TV-G' 'TV-MA' 'TV-PG' 'TV-Y' 'TV-Y7' 'Unrated' 'X']
content_rating
Approved       55
G             112
GP              6
M               5
NC-17           7
Not Rated     116
PG            701
PG-13        1461
Passed          9
R            2118
TV-14          30
TV-G           10
TV-MA          20
TV-PG          13
TV-Y            1
TV-Y7           1
Unrated        62
X              13
Name: movie_title, dtype: int64


In [12]:
from bokeh.charts import BoxPlot
b = BoxPlot(data.dropna(how="any", subset=["content_rating", "imdb_score"]), 
            values="imdb_score", label="content_rating", tools=tools)
show(b)

In [15]:
b = BoxPlot(data.dropna(how="any", subset=["content_rating", "budget"]), \
            values="budget", label="content_rating", tools=tools)
show(b)

Si hacemos el boxplot sobre el número de votos, ¿qué nos indica el gráfico resultante sobre la distribución de votos en IMDB?

In [21]:
from bokeh.charts import Bar
# stack is a grouping feature
br = Bar(data[data["title_year"]>1990].dropna(how="any", subset=["content_rating"]), 
         label="title_year", 
         values="title_year", stack="content_rating", agg="count", tools=tools, legend="top_right" )
show(br)

## Bokeh y Big Data

En lo visto hasta aquí, los datos están en memoria (DataFrame de pandas) y Bokeh los utiliza íntegramente, lo cual deja de ser práctico cuando hay muchos datos a mostrar en un solo gráfico. 

Bokeh integra la posibilidad de utilizar [WebGL](http://bokeh.pydata.org/en/latest/docs/user_guide/webgl.html#userguide-webgl) para acelerar mediante la Graphics Processing Unit (GPU) el javascript generado. Esto simplemente aprovecha una característica presente en la mayoría de los navegadores modernos.
  

Otra posibilidad es utilizar un Servidor de Bokeh, de modo que los gráficos se actualizan en el servidor y los clientes JavaScript proporcionan una vista.

![Bokeh Server](http://bokeh.pydata.org/en/0.8.2/_images/bokeh_server.png)

Por último, Bokeh se puede combinar con proyectos como [Blaze](http://blaze.readthedocs.org/en/latest/index.html), una biblioteca que trata de proporcionar una interfaz de programación parecida a los DataFrames de pandas, pero actuando sobre conjuntos de datos grandes, que en ocasiones no se pueden tener en memoria.

https://www.youtube.com/watch?v=mC2uEUyeNWQ